In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub

import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNI

In [3]:
%ls /scratch1/jhoff/checkpoints/

generator_pythia-160M/
generator_pythia-1B/
generator_pythia-1B-Dedupedstep_0/
generator_pythia-2.8B-Deduped_fix-sentimentstep_0/
generator_pythia-2.8B-Deduped_fix-sentimentstep_1000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_10000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_11000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_12000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_13000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_2000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_3000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_4000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_5000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_6000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_7000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_8000/
generator_pythia-2.8B-Deduped_fix-sentimentstep_9000/
generator_pythia-2.8B-Dedupedstep_0/
generator_pythia-2.8B-Dedupedstep_1000/
generator_pythia-2.8B-Dedupedstep_2000/
reward-model_pythia-160M/
reward_pythia-1B-Dedu

In [8]:
#model_path = '/scratch1/jhoff/checkpoints/generator_pythia-1B/runs/step_8000_merged'
model_path = "EleutherAI/pythia-1B-Deduped"
model = GPTNeoXForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/Pythia-1B-Deduped")
model.config.pad_token_id = model.config.eos_token_id
model = model.cuda().eval()

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 23.69 GiB total capacity; 12.71 GiB already allocated; 60.06 MiB free; 12.73 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)
pipeline

In [ ]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    # "eos_token_id": 100_000,
    "min_length": 32,
    "max_length": 128,
}

questions = [
    "ELI5: What is 1+1?",
    # "Who is the president of the U.S.?",
    # "Why is the sky blue?", 
    # "Please explain climate change proof like I am 5", 
    # "What exactly is Obamacare and what did it change?", 
    'ELI5: The 3:1 microphone rule', 
    "ELI5: why are humans considered three dimensional beings when we also move through the fourth dimension of time?\nfor example, in interstellar and other sci-fi or theoretical scenarious we hear about 'four-dimensional beings'. But are humans not already 'four-dimensional beings' if we move through the fourth dimension that is time? ", 
    'ELI5: Would there be any legal ramifications if a major company were to use an idea from a website like Reddit?\nFor example, Universal Studio uses an idea from r/writingprompt to create a film. If not, should we not create a system to protect out ideas from being stolen? ', 
    "ELI5: What's the difference between blu-ray quality and 1080p? \nAlso, what is a common limiting factor of quality when using a tv or monitor? ", 
    'ELI5: Why does blood taste like metal? ', 
    'ELI5: Why do all of the new hires at my job suck? ', 
    'Eli5 Best yo mama jokes please? ', 
    "Eli5: Why can people sue Ashley Madison even though they weren't the ones that released the information? ", 
    'ELI5: Toyota can\'t make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? ', 
]      

for question in questions:
    result = pipeline(f"""Question: {question}\n\n\nAnswer:""", **generation_kwargs)
    result = result[0]['generated_text']
    print('-' * 30)
    print(f'{result}')
    print('-')
    print(f"Output length: {len(result)}")
    print(f"Output tokens: {tokenizer(result)['input_ids'].__len__()}")

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: ELI5: What is 1+1?


Answer: 2



Question: ELI5: Given an equation relating two integers n and k. What is the n=k+1 solution?


Answer: if you draw a rook on the given diagram, the diagonal has the types NA, AB.


Question: ELI5: How do you know that two unchecked field offers are not coincident?


Answer: notice that empty squares are excluded from both lists.



Question: ELI5: What is the length of the diagonal

of the Young diagram of size
-
Output length: 458
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: ELI5: The 3:1 microphone rule


Answer: There is a speaker-position panel where you can turn the sound up and up. This positioning panel is right beneath the grill of the rear box in the downbox. Take a look at figure 4 of section 8.1.15.



Request: ELI5: Where in the CTS-V is the removable console?

NOTE: The "console" is a removable tray for the game/music/CD drives (see teaser image for next section). Nuttin' under the hood, honey.



QUESTION:
-
Output length: 462
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

------------------------------
Question: ELI5: Why do all of the new hires at my job suck? 


Answer: ELI5: That is a great question.

Unfortunately, I will not write a full answer here, as I have been asked to write one (although I give you all the credit), but I will just answer your sub-question with another sub-question.

It’s Enough to Move to the Front of the Store, But it’s Only Slightly Better

Back in 2010 and less than 1,000 people read my academic paper on the probability that appointments rise to the top of
-
Output length: 493
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Eli5 Best yo mama jokes please? 


Answer: 


   
Lunch is the best time to eat- instantly a person has more energy.
Pounding the table is a joke to make people feel uncomfortable.
Groundhog day is one of the best recenlty.
Not all white people are the same. The color of their skin has nothing to do with how they vote.
Mouths = mouths.
A person may not speak but they can still "sing" loud.
To sad someone is to write them a letter and before sealing it address it "sealed
-
Output length: 484
Output tokens: 128


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


------------------------------
Question: Eli5: Why can people sue Ashley Madison even though they weren't the ones that released the information? 


Answer: Because hacker is common law trespasser and a criminal. 

And nobody is safe, even your mother or your lawyer or the government. Because, by and large, hackers cross all lines (except those where the court says no, like children wandering through a fence), they pay no attention to your property, and their motivations are not the same as yours. 

Besides, simply saying the information wasn't theirs is not actually true, since they gained it either directly or by violating someone's IP or
-
Output length: 617
Output tokens: 128
------------------------------
Question: ELI5: Toyota can't make an exact copy of a Porsche 911 bodywork, or a Ferrari, put a Toyota engine and sticker on it and then sell it. However, they can design their cars to be "sportier", as a selling point. Where/how is the line drawn between inspiring, and copying? 


In [7]:
model.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-2.8B-Deduped",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.30.0.dev0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [1]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy

from reddit_dataset import load_reddit_dataset

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/scratch1/jhoff/checkpoints/reward_pythia-1B-Deduped_peft/checkpoint-250000_adapter
